In [1]:
# ensemble inference pipeline

# TODO
# locate test video folder
# set test frame folder that will contain extracted face frames
# gather existing files from frame folder

# if all of a certain video's frame is in the folder
# skip that video
# else extract frames from each video

# create CustomModule.py or CustomModules folder

# when frames are ready
# each models take turn and runs inference
# result of inference is stored in padas dataframe

# ensemble( choose between hard / soft voting)
# if save sample_submission, save 
# else save as results.pickle

#TODO
# face detection py 20
# relocate weights for facedet

In [3]:
# import
!pip install efficientnet_pytorch

import os

import pandas as pd

from face_detection import face_extractor

Using legacy 'setup.py install' for efficientnet-pytorch, since package 'wheel' is not installed.
    Running setup.py install for efficientnet-pytorch: started
    Running setup.py install for efficientnet-pytorch: finished with status 'done'


You should consider upgrading via the 'C:\Users\Admin\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


ModuleNotFoundError: No module named 'torchvision'

In [3]:
TEST_PATH = '/mnt/elice/dataset/test'
FRAME_FOLDER = '/home/elicer/data/test/test'

In [4]:
# make directories to save frames
os.makedirs(FRAME_FOLDER, exist_ok = True)

In [5]:
# parameters
frames_per_video = 40
real_fake_threshold = .6

In [ ]:
# run face extractrion
face_extractor(video_path = TEST_PATH,
               save_path = FRAME_FOLDER, 
               frames_per_video = frames_per_video, 
               skip_exist=True)

In [ ]:
# run model inference
!python test_model.py \
--net EfficientNetB4 \
--face scale \
--size 224 \
--testsets /home/elicer/data/test/test_df.pkl \
--model_path /home/elicer/weights/binclass/temporary_tag4/bestval.pth \
--batch 16 \
--result_path /home/elicer/results/kwon_result.pkl

# re-declare parameters
modelname_result_path = '/home/elicer/results/kwon_result.pkl'


In [ ]:
# run model inference
!python test_model.py \
--net EfficientNetAutoAttB4 \
--face scale \
--size 224 \
--testsets /home/elicer/data/test/test_df.pkl \
--model_path /home/elicer/weights/binclass/choi/bestval.pth \
--batch 16 \
--result_path /home/elicer/results/choi_result.pkl

# re-declare parameters
modelname_result_path2 = '/home/elicer/results/choi_result.pkl'


In [ ]:
# run model inference
!python test_model.py \
--net EfficientNetB4 \
--face scale \
--size 224 \
--testsets /home/elicer/data/test/test_df.pkl \
--model_path /home/elicer/weights/binclass/s_in/it001500.pth \
--batch 16 \
--result_path /home/elicer/results/s_in_result.pkl

# re-declare parameters
modelname_result_path3 = '/home/elicer/results/s_in_result.pkl'


In [ ]:
# load results and vote

# load result df 1
df = pd.read_pickle(modelname_result_path)
# groupby and mean
df['path'] = df['face_path'].apply(lambda x: x.split('/')[-1].split('_')[0]+'.mp4')
df = df.drop(columns=['face_path','label'])
result_df_1 = df.groupby(['path']).mean().reset_index()

# load result df 2
df = pd.read_pickle(modelname_result_path2)
# groupby and mean
df['path'] = df['face_path'].apply(lambda x: x.split('/')[-1].split('_')[0]+'.mp4')
df = df.drop(columns=['face_path','label'])
result_df_2 = df.groupby(['path']).mean().reset_index()

# load result df 3
df = pd.read_pickle(modelname_result_path3)
# groupby and mean
df['path'] = df['face_path'].apply(lambda x: x.split('/')[-1].split('_')[0]+'.mp4')
df = df.drop(columns=['face_path','label'])
result_df_3 = df.groupby(['path']).mean().reset_index()

results = pd.merge(pd.merge(result_df_1,result_df_2, on='path'),result_df_3, on='path')
results['avg_score'] = results[['score_x','score_y','score']].mean(axis=1)

# ensemble( choose between hard / soft voting)
# if save sample_submission, save 
# else save as results.pickle

sample_sub = pd.read_csv('/home/elicer/sample_submission.csv', usecols=['path'])
sample_sub = sample_sub.join(results[['path','avg_score']], how='left',rsuffix='tt').drop(columns=['pathtt'])

sample_sub['avg_score'] = sample_sub['avg_score'].fillna(0)
sample_sub['label'] = sample_sub['avg_score'].apply(lambda x : 'fake' if x < real_fake_threshold else 'real')

sample_sub.drop('avg_score',axis=1).to_csv('/home/elicer/sample_submission.csv',index=False)
